# 多项式核 SVM 分类器

## 核技巧原理

多项式核通过核函数隐式计算高维特征空间中的内积，无需显式构造高维特征。

### 核函数定义

$$K(x, x') = (\gamma x^\top x' + r)^d$$

- $d$: 多项式阶数 (degree)
- $\gamma$: 缩放系数 (gamma)
- $r$: 独立项 (coef0)，控制高阶与低阶项权重

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# 生成数据
X, y = make_moons(n_samples=200, noise=0.15, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"训练集: {len(X_train)}, 测试集: {len(X_test)}")

In [ ]:
# 多项式核 SVM
poly_kernel_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svm_clf', SVC(kernel='poly', degree=3, coef0=1, C=5, random_state=42))
])

poly_kernel_svm.fit(X_train, y_train)

# 评估
train_acc = accuracy_score(y_train, poly_kernel_svm.predict(X_train))
test_acc = accuracy_score(y_test, poly_kernel_svm.predict(X_test))

print(f"训练准确率: {train_acc:.4f}")
print(f"测试准确率: {test_acc:.4f}")
print(f"支持向量数: {poly_kernel_svm.named_steps['svm_clf'].n_support_}")

In [ ]:
# 可视化决策边界
def plot_boundary(model, X, y, ax, title):
    h = 0.02
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
    ax.scatter(X[y==0, 0], X[y==0, 1], c='steelblue', s=40, edgecolors='white')
    ax.scatter(X[y==1, 0], X[y==1, 1], c='coral', s=40, edgecolors='white')
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

fig, ax = plt.subplots(figsize=(10, 6))
plot_boundary(poly_kernel_svm, X, y, ax, f'多项式核 SVM (degree=3, Acc={test_acc:.2%})')
plt.tight_layout()
plt.show()

In [ ]:
# 比较不同阶数和 coef0
params = [(2, 0), (2, 1), (3, 0), (3, 1), (5, 0), (5, 1)]
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (d, c0) in enumerate(params):
    model = Pipeline([('scaler', StandardScaler()),
                      ('svm', SVC(kernel='poly', degree=d, coef0=c0, C=5))])
    model.fit(X_train, y_train)
    acc = accuracy_score(y_test, model.predict(X_test))
    plot_boundary(model, X, y, axes[idx], f'd={d}, coef0={c0}\nAcc={acc:.2%}')

plt.suptitle('多项式核: degree 和 coef0 的影响', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("coef0 影响:")
print("- coef0=0: 齐次多项式，只有高阶项")
print("- coef0>0: 非齐次多项式，包含低阶项")

In [ ]:
# 网格搜索
param_grid = {'svm__degree': [2, 3, 4], 'svm__C': [1, 10], 'svm__coef0': [0, 1]}
grid = GridSearchCV(Pipeline([('scaler', StandardScaler()), ('svm', SVC(kernel='poly'))]),
                    param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)

print(f"最佳参数: {grid.best_params_}")
print(f"最佳 CV 准确率: {grid.best_score_:.4f}")
print(f"测试准确率: {accuracy_score(y_test, grid.predict(X_test)):.4f}")

In [ ]:
# 单元测试
def run_tests():
    tests = []
    try:
        m = Pipeline([('s', StandardScaler()), ('svm', SVC(kernel='poly', degree=3))])
        m.fit(X_train, y_train)
        tests.append(("模型训练", True, ""))
    except Exception as e:
        tests.append(("模型训练", False, str(e)))
    
    try:
        pred = m.predict(X_test)
        assert pred.shape == y_test.shape
        tests.append(("预测输出", True, ""))
    except Exception as e:
        tests.append(("预测输出", False, str(e)))
    
    try:
        acc = accuracy_score(y_test, pred)
        assert acc > 0.8
        tests.append(("准确率", True, f"{acc:.2%}"))
    except Exception as e:
        tests.append(("准确率", False, str(e)))
    
    print("="*50 + "\n单元测试结果\n" + "="*50)
    for name, ok, msg in tests:
        print(f"{'✓' if ok else '✗'} {name}" + (f" ({msg})" if msg else ""))
    print(f"通过: {sum(t[1] for t in tests)}/{len(tests)}")

run_tests()

## 总结

### 多项式核要点
- **degree**: 多项式阶数，越高模型越复杂
- **coef0**: 独立项，影响高/低阶项权重
- **优势**: 隐式高维映射，避免特征爆炸
- **适用**: 数据有明显多项式结构时

### 与显式多项式特征对比
| 方法 | 特征空间 | 计算效率 | 内存占用 |
|------|----------|----------|----------|
| 显式特征 | 显式构造 | 低 | 高 |
| 核技巧 | 隐式计算 | 高 | 低 |